In [26]:
import numpy as np
from numpy import diff
from numpy import sum
from random import randint
import math
import time

np.__version__

'1.26.4'

In [27]:
from enum import Enum

class State(Enum):
    SECOS_1 = 1
    DADOS_1 = 2
    SECOS_2 = 3
    DADOS_2 = 4
    TABELA = 5
    FIM = 6

In [41]:
class Yan:
    def __init__(self):
        # iniciar a coluna da desordem com -1 para indicar que as celulas estão vazias
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1, "x+": -1, "x-": -1, "y": -1}
        self.yangame = "YanGame"
        self.dices = [0,0,0,0,0]
        self.current_state = State.SECOS_1
        self.new_dice = list()
        self.marcado_em = ""
        self.over_minimum = False
        self.is_ended = False
        self.game_play = []
        self.next_state = self.get_current_state()
        self.reward = 0
        

        # self.action_size = self.row_count * self.column_count
    
    def check_consecutive(self, l):
        n = len(l) - 1
        return sum(diff(sorted(l)) == 1) >= n


    def roll_dice(self, n):
        rolls = list()
        for i in range(n):
            rolls.append(randint(1, 6))
        rolls.sort()
        self.game_play.append("você rolou:                          " + str(rolls))
        return rolls


    def get_table_state(self):
        state = "5"
        # estado, dices, table
        for die in self.dices:
            state += str(die)
         
        binary_table = ""
        for item in list(self.desordem.values()):
            if item == -1:
                binary_table += "0"
            else:
                binary_table += "1"
        state += str(binary_table)
        # print(state)
        if state == "":
            breakpoint()
        return int(state)


    def get_current_state(self):
        state = ""
        # estado, cur die, dices, table, wr
        state += str(self.current_state.value)

        for die in self.dices:
            state += str(die)
         
        for item in self.desordem:
            value = "0"
            if self.desordem[item] != -1:
                value = "1"
            if item == "q" or item == "f" or item == "y" or item == "x+" or item == "x-":
                value = ""
            state += value
        
        if state == "":
            breakpoint()
        return int(state)

    # onde é possivel anotar o resultado dos dados
    def get_empty_cells(self):
        # checar quais celulas estão vazias
        avaiable_list = {}
        for item in self.desordem:
            if self.desordem[item] == -1:
                avaiable_list[item] = self.desordem[item]
        return avaiable_list
    
    def get_number_of_actions(self):
        if self.current_state == State.TABELA:
            # se estivermos na tabela, retornamos quantidade de celulas em branco
            return len(self.get_empty_cells())
        elif self.current_state == State.SECOS_1 or self.current_state == State.SECOS_2:
            #se tivermos no SECO retornamos binário "marcar/ não marcar"
            return 2
        else:
            # se estivermos nos dados retornamos 31 combinações de dados
            return 31


    def do_the_reroll(self, n_dices):
        dices_to_reroll = '{0:05b}'.format(int(n_dices))
        self.game_play.append("dados a serem rolados novamente:     " + dices_to_reroll)
        for index in range(len(str(dices_to_reroll))):
            if dices_to_reroll[index] == "1":
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.game_play.append("seus dados ficaram assim:            " + str(self.dices))
        dice_set = set(self.dices)
        n_single = len(dice_set)
        # return (6 - n_single) * 3
        return 0


    def reset(self):
        self.game_play = []
        self.game_play.append("Jogo iniciado")
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1,
                         "x+": -1, "x-": -1,
                         "y": -1}
        self.current_state = State.SECOS_1
        self.dices = self.roll_dice(5)
        self.dices.sort()
        self.is_ended = False

        return self.get_current_state()
    

    def get_initial_state(self):
        return self.reset()
    

    def get_valid_moves(self, state):
        game_state = int(str(state)[:1])       
        if game_state == State.TABELA.value:
            # se estivermos na tabela, retornamos quantidade de celulas em branco
            return np.ones(len(self.get_empty_cells()))
        elif game_state == State.SECOS_1.value or game_state == State.SECOS_2.value:
            #se tivermos no SECO retornamos binário "marcar/ não marcar"
            return np.ones(2)
        else:
            # se estivermos nos dados retornamos 31 combinações de dados
            return np.ones(31)
        # return self.get_number_of_actions()
        # return (state.reshape(-1) == 0).astype(np.uint8)
    

    def calculate_reward(self, reward, cell, times_over):
        return reward
        if cell == "1" or "2" or "3" or "4" or "5" or "6":
            total_upper = sum(list(self.desordem.values())[0:6])
            # if total_upper >= 60:
            #     return 300

            if self.over_minimum:
                return reward
            else:
                return 0


    def set_cell_value(self, cell: str):
        
        self.game_play.append("você marcou                          " + str(self.dices) + " em " + cell)
        self.game_play.append("\n")
        
        # self.marcado_em = 
        # print(self.marcado_em)
        points = 0

        match cell:
            case "1":
                points = self.dices.count(1)
                self.desordem["1"] = points
                self.over_minimum = self.dices.count(1) > 1
                return self.calculate_reward(points, cell, self.dices.count(1))
                
            case "2":
                points = self.dices.count(2) *2
                self.desordem["2"] = points
                self.over_minimum = self.dices.count(2) > 1
                return self.calculate_reward(points, cell, self.dices.count(2))
            case "3":
                points = self.dices.count(3) * 3
                self.desordem["3"] = points
                self.over_minimum = self.dices.count(3) > 2
                return self.calculate_reward(points, cell, self.dices.count(3))
            case "4":
                points = self.dices.count(4) * 4
                self.desordem["4"] = points
                self.over_minimum = self.dices.count(4) > 2
                return self.calculate_reward(points, cell, self.dices.count(4))
            case "5":
                points = self.dices.count(5) * 5
                self.desordem["5"] = points
                self.over_minimum = self.dices.count(5) > 2
                return self.calculate_reward(points, cell, self.dices.count(5))
            case "6":
                points = self.dices.count(6) * 6
                self.desordem["6"] = points
                self.over_minimum = self.dices.count(6) > 2
                return self.calculate_reward(points, cell, self.dices.count(6))
            case "y":
                if self.dices[0] == self.dices[4]:
                    self.desordem["y"] = sum(self.dices) + 50
                    return sum(self.dices) + 50
                else:
                    self.desordem["y"] = 0
                    return 0
            case "q":
                if self.dices[0] == self.dices[3]:
                    self.desordem["q"] = sum(self.dices[:4]) + 30
                    return sum(self.dices[:4]) + 30
                elif self.dices[1] == self.dices[4]:
                    self.desordem["q"] = sum(self.dices[1:]) + 30
                    return sum(self.dices[1:]) + 30
                else:
                    self.desordem["q"] = 0
                    return 0
            case "f":
                if (self.dices[0] == self.dices[2] and self.dices[3] == self.dices[4]) or (
                        self.dices[0] == self.dices[1] and self.dices[2] == self.dices[4]):
                    self.desordem["f"] = sum(self.dices) + 20
                    return sum(self.dices) + 20
 
                else:
                    self.desordem["f"] = 0
                    return 0
            case "s+":
                if self.check_consecutive(self.dices) and self.dices[0] == 2:
                    self.desordem["s+"] = 60
                    return 60
                else:
                    self.desordem["s+"] = 0
                    return 0
            case "s-":
                if self.check_consecutive( self.dices) and self.dices[0] == 1:
                    self.desordem["s-"] = 50
                    return 50
                else:
                    self.desordem["s-"] = 0
                    return 0
            case "x+":
                if (sum(self.dices) > self.desordem["x-"]) or self.desordem["x-"] == -1:
                    self.desordem["x+"] = sum(self.dices)
                    return sum(self.dices)
                else:
                    self.desordem["x+"] = 0
                return 0
            case "x-":
                if (sum(self.dices) < self.desordem["x+"]) or self.desordem["x+"] == -1:
                    self.desordem["x-"] = sum(self.dices)
                    return sum(self.dices)
                else:
                    self.desordem["x-"] = 0
                return 0


    def get_next_state(self, state, action):
        action = int(action)
        game_state = int(str(state)[:1])       
        if game_state == int(State.SECOS_1.value):
            if action:
                self.game_play.append("você escolheu marcar")
                self.current_state = State.TABELA
                self.next_state = self.get_table_state()
                return (self.next_state, 0, self.is_ended)

            else:
                self.game_play.append("você escolheu jogar novamente")
                self.current_state = State.DADOS_1
                self.next_state = self.get_current_state()
                return (self.next_state, 0, self.is_ended)
        
        elif game_state == int(State.DADOS_1.value):
            self.reward = self.do_the_reroll(n_dices=(int(action)+1))
            self.current_state = State.SECOS_2
            self.next_state = self.get_current_state()
            return (self.next_state, self.reward, self.is_ended)
        
        elif game_state == int(State.SECOS_2.value):
            if action:
                self.game_play.append("você escolheu marcar")
                self.current_state = State.TABELA
                self.next_state = self.get_table_state()
                return (self.next_state, 0, self.is_ended)

            else:
                self.game_play.append("você escolheu jogar novamente")
                self.current_state = State.DADOS_2
                self.next_state = self.get_current_state()
                return (self.next_state, 0, self.is_ended)
        
        elif game_state == int(State.DADOS_2.value):
            self.reward = self.do_the_reroll(n_dices=(action+1))
            self.current_state = State.TABELA
            self.next_state = self.get_table_state()
            return (self.next_state, self.reward, self.is_ended)
        
        elif game_state == int(State.TABELA.value):
            avaiable_cells = list(self.get_empty_cells().keys())
            table_index = avaiable_cells[action]
            self.game_play.append("suas opções são:                     " + str(avaiable_cells))

            self.reward = self.set_cell_value(table_index)
            print(self.desordem)
            if self.check_ended():
                self.game_play.append("fim do jogo")
                self.current_state = State.FIM
                self.next_state = self.get_current_state()
                return (self.next_state, self.reward, self.is_ended)
            else:
                self.is_ended = False
                self.current_state = State.SECOS_1
                self.dices = self.roll_dice(5)
                self.dices.sort()
                self.next_state = self.get_current_state()
                return (self.next_state, self.reward, self.is_ended)
            
        else:
            self.is_ended = True
            return (self.get_current_state(), self.get_total_score(), self.is_ended)

        return state
    

    def check_ended(self):
        return -1 not in self.desordem.values()
    
    def get_value_and_terminated(self, state, action):
        return self.get_total_score(), self.check_ended()
    

    def get_total_score(self):
        self.score_values = list(self.desordem.values())
        self.total = sum(self.score_values)
        total_upper = sum(self.score_values[0:6])
        if total_upper >= 60:
            self.total += 30
        # print()
        self.game_play.append("sua tabela ficou assim:              " + str(self.desordem))
        self.game_play.append("TOTAL:                               " + str(self.total))
        print("total: ", self.total)
        print("sua tabela ficou assim: ", self.desordem)
        print()
        

        return self.total

    

In [29]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        
        self.children = []
        self.expandable_moves = game.get_valid_moves(state)
        
        self.visit_count = 0
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)
    
    def expand(self):
        action = int(np.random.choice(self.expandable_moves))
        self.expandable_moves[action] = 0
        
        child_state = int(str(self.state))
        child_state = self.game.get_next_state(child_state, action)
        # child_state = self.game.change_perspective(child_state, player=-1)
        
        child = Node(self.game, self.args, child_state[0], self, action)
        self.children.append(child)
        return child
    
    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        # value = self.game.get_opponent_value(value)
        
        if is_terminal:
            return value
        
        rollout_state = int(str(self.state))
        # rollout_state = self.state.copy()
        # rollout_player = 1
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(len(valid_moves))
            rollout_state = self.game.get_next_state(rollout_state, action)[0]
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            if is_terminal:
                return value    
            
            
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.reward
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args
        
    def search(self, state):
        root = Node(self.game, self.args, state)
        
        for search in range(self.args['num_searches']):
            node = root
            
            while node.is_fully_expanded():
                node = node.select()
                
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            # value = self.game.get_opponent_value(value)
            
            if not is_terminal:
                node = node.expand()
                value = node.simulate()
                
            node.backpropagate(value)    
            
            
        action_probs = self.game.get_valid_moves(state)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs
        
        
        

In [42]:
game = Yan()

args = {
    'C': 1.41,
    'num_searches': 1000
}

mcts = MCTS(game, args)

state = game.get_initial_state()


while True:
    print(state)
    
    mcts_probs = mcts.search(state)
    action = np.argmax(mcts_probs)
        
    state = game.get_next_state(state, action)
    
    value, is_terminal = game.get_value_and_terminated(state, action)
    
    if is_terminal:
        print(game.desordem)
        print(game.get_total_score())
        break
        
    # player = game.get_opponent(player)


12335600000000
total:  -13
sua tabela ficou assim:  {'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': -1, 'q': -1, 'f': -1, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}

total:  -13
sua tabela ficou assim:  {'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': -1, 'q': -1, 'f': -1, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}

{'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': 6, 'q': -1, 'f': -1, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}
total:  -6
sua tabela ficou assim:  {'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': 6, 'q': -1, 'f': -1, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}

total:  -6
sua tabela ficou assim:  {'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': 6, 'q': -1, 'f': -1, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}

{'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': -1, 's-': -1, 'x+': -1, 'x-': -1, 'y': -1}
total:  -5
sua tabela ficou assim:  {'1': -1, '2': -1, '3': -1, '4': -1, '5': -1, '6': 6, 'q': -1, 'f': 0, '

total:  19
sua tabela ficou assim:  {'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': -1}

{'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}
total:  20
sua tabela ficou assim:  {'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}

total:  20
sua tabela ficou assim:  {'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}

total:  20
sua tabela ficou assim:  {'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}

total:  20
sua tabela ficou assim:  {'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': -1, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}

{'1': 1, '2': -1, '3': -1, '4': 0, '5': -1, '6': 6, 'q': 0, 'f': 0, 's+': 0, 's-': -1, 'x+': -1, 'x-': 19, 'y': 0}
tot